In [ ]:

import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
from google.colab import drive
drive.mount('content/')


Mounted at content/


In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models # для подгрузки моделей TF

Cloning into 'models'...
remote: Enumerating objects: 2802, done.
remote: Counting objects: 100% (2802/2802), done.
remote: Compressing objects: 100% (2335/2335), done.
remote: Total 2802 (delta 716), reused 1285 (delta 431), pack-reused 0
Receiving objects: 100% (2802/2802), 32.79 MiB | 31.94 MiB/s, done.
Resolving deltas: 100% (716/716), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

# загружаем и устанавливаем зависимости

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1658741 sha256=7b00af84a1265db8b6e9deea259e5839313fd4df88a915fcb06ac1d3cc48d60f
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3bkzwwu/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=466c12f3e2a11ff7f6cf1ade580144b949cd4c8d495bb4d3880f2043fc6be6ae
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=5842b7be2affb514bce41f5fbd53edbc08cb609279786a60e3389abfab7a2448
  Stored in directory: /root/



ERROR: multiprocess 0.70.12.2 has requirement dill>=0.3.4, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.26.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.31.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils
from object_detection.utils import ops as utils_ops

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz' # название файла с моделью
  model_dir = tf.keras.utils.get_file( # директория модели
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model" 
  model = tf.saved_model.load(str(model_dir)) # загружаем модель
  model = model.signatures['serving_default']

  return model

In [ ]:
# для добавления метки к bounding box
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images') # директория тестовых картинок
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('models/research/object_detection/test_images/image1.jpg'),
 PosixPath('models/research/object_detection/test_images/image2.jpg'),
 PosixPath('models/research/object_detection/test_images/image3.jpg')]

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17' # тип модели для загрузки
detection_model = load_model(model_name) # загружаем модель

76537856/76534733 [==============================] - 1s 0us/step


In [ ]:
print(detection_model.inputs) # посмотрим на входы-выходы модели
detection_model.output_dtypes

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


{'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32}

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image) # переводим в тензор
  input_tensor = input_tensor[tf.newaxis,...] # расширяем размерность, для передачи батчей

  output_dict = model(input_tensor) # запуск модели
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                for key,value in output_dict.items()} # переведем в numpy array и возтмем первый элемент
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64) # переводим предсказания в целочисленные

  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
              image.shape[0], image.shape[1]) # подгоняем bounding box к размеру картинки  
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                      tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  image_np = np.array(Image.open(image_path)) # загружаем картинку как numpy array
  output_dict = run_inference_for_single_image(model, image_np) # делаем детекцию
  visualization_utils.visualize_boxes_and_labels_on_image_array( # визуализируем результаты детекции
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

Output hidden; open in https://colab.research.google.com to view.